In [1]:
! pip install -q pyspark==3.1.2 spark-nlp

In [2]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-04-20 04:09:01--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-04-20 04:09:02--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-04-20 04:09:02--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from sparknlp.base import *
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
import pandas as pd 

import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Suspicious Communication on Social Platforms.csv to Suspicious Communication on Social Platforms (2).csv


In [ ]:
df_cybu = pd.read_csv('Suspicious Communication on Social Platforms.csv')


In [ ]:
df = pd.DataFrame(df_cybu, columns = ['comments', 'tagging'])
df.head()

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1


In [ ]:
import string

In [ ]:
def remove_pun(text):
  for pun in string.punctuation:
    text = text.replace(pun,"" )
  text = text.lower()
  return(text)  

df['Text']=df['comments'].apply(remove_pun)



In [ ]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [ ]:
data = sqlContext.createDataFrame(df)
print(data.printSchema())

root
 |-- comments: string (nullable = true)
 |-- tagging: long (nullable = true)
 |-- Text: string (nullable = true)

None


In [ ]:
sp = SparkSession(sc)

In [ ]:
spark_df = sp.createDataFrame(df.astype(str))

In [ ]:
spark_df.show()


+--------------------+-------+--------------------+
|            comments|tagging|                Text|
+--------------------+-------+--------------------+
| Get fucking real...|      1| get fucking real...|
| She is as dirty ...|      1| she is as dirty ...|
| why did you fuck...|      1| why did you fuck...|
| Dude they dont f...|      1| dude they dont f...|
| WTF are you talk...|      1| wtf are you talk...|
|Ill save you the ...|      1|ill save you the ...|
| Im dead serious....|      1| im dead seriousr...|
|...go absolutely ...|      1|go absolutely ins...|
|Lmao  im watching...|      1|lmao  im watching...|
|LOL  no he said  ...|      1|lol  no he said  ...|
|truth on both cou...|      1|truth on both cou...|
|   Shakespeare nerd!|      1|    shakespeare nerd|
|you are SUCH a fu...|      1|you are such a fu...|
|Heh. Fuck 'em WHE...|      1|   heh fuck em where|
|damn it i totally...|      1|damn it i totally...|
|wow  damn I would...|      1|wow  damn i would...|
|nigga u gei

In [ ]:
spark = sparknlp.start()

In [ ]:
documentAssembler = DocumentAssembler()\
.setInputCol("Text")\
.setOutputCol("document")


In [ ]:
doc_df=documentAssembler.transform(spark_df)
doc_df.show()

In [ ]:
doc_df.printSchema()

In [ ]:
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")

In [ ]:
doc_df.withColumn(
    "tmp", 
    F.explode("document")) \
    .select("tmp.*"). \
    show()

In [ ]:
(train, test) = doc_df.randomSplit([0.8, 0.2])


In [ ]:
dtc = DecisionTreeClassifier(featuresCol="Text", labelCol="tagging")
dtc = dtc.fit(train)

In [ ]:
pred = dtc.transform(test)
pred.show()

,comments,tagging,Text,result
0,Get fucking real dude.,1,get fucking real dude,1
1,She is as dirty as they come and that crook ...,1,she is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1,why did you fuck it up i could do it all day ...,0
3,Dude they dont finish enclosing the fucking s...,1,dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1,wtf are you talking about men no men thats no...,0
5,Ill save you the trouble sister. Here comes a ...,1,ill save you the trouble sister here comes a b...,1
6,Im dead serious.Real athletes never cheat don...,1,im dead seriousreal athletes never cheat dont...,0
7,...go absolutely insane.hate to be the bearer ...,1,go absolutely insanehate to be the bearer of b...,1
8,Lmao im watching the same thing ahaha. The ga...,1,lmao im watching the same thing ahaha the gay...,1
9,LOL no he said What do you call a jail cell ...,1,lol no he said what do you call a jail cell ...,1


In [ ]:
sc.stop()